In [1]:
pip install torch --upgrade

In [3]:
pip install opencv-python torch

Note: you may need to restart the kernel to use updated packages.


In [143]:
!pip install -U ultralytics

In [10]:
import cv2
import cvzone
import math
from ultralytics import YOLO

cap = cv2.VideoCapture(0) 
cap.set(3, 1280) 
cap.set(4, 720) 

model = YOLO('yolov8n.pt')

# classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat", "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat", "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat", "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed", "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"]

alert_flag = False

while True:
    success, img = cap.read()
    
    if not success:
        print("Failed to read from camera")
        break  # Exit the loop if reading from the camera fails
        
    # Flip the frame horizontally to correct lateral inversion
    img = cv2.flip(img, 1)

    results = model(img, stream=True)
    for r in results:
        boxes = r.boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            w, h = x2 - x1, y2 - y1
            cvzone.cornerRect(img, (x1, y1, w, h))

            # conf = math.ceil((box.conf[0] * 100)) / 100

            # cls = box.cls[0]
            # name = classNames[int(cls)]

            # cvzone.putTextRect(img, f'Plant', f'{conf}', (max(0, x1), max(35, y1)), scale=0.5)
            # cvzone.putTextRect(img, f'{name} 'f'{conf}', (max(0, x1), max(35, y1)), scale=0.5)
            
            # Calculate the center region based on frame size
            frame_height, frame_width, _ = img.shape
            center_x_min = int(frame_width * 0.4)
            center_x_max = int(frame_width * 0.6)
            center_y_min = int(frame_height * 0.4)
            center_y_max = int(frame_height * 0.6)

            # Check if the object is near the dynamically calculated center
            if center_x_min < (x1 + x2) / 2 < center_x_max and center_y_min < (y1 + y2) / 2 < center_y_max:
                alert_flag = True
            else:
                alert_flag = False

    if alert_flag:
        cv2.putText(img, "Plant is in Center!", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        print("Plant is in Center!")

    cv2.imshow("Image", img)

    # Exit if the user presses 'q'
    key = cv2.waitKey(1)
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 384x640 1 person, 123.7ms
Speed: 4.0ms preprocess, 123.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Plant is in Center!

0: 384x640 1 person, 131.5ms
Speed: 2.5ms preprocess, 131.5ms inference, 4.4ms postprocess per image at shape (1, 3, 384, 640)
Plant is in Center!

0: 384x640 1 person, 108.0ms
Speed: 9.5ms preprocess, 108.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Plant is in Center!

0: 384x640 1 person, 115.2ms
Speed: 2.5ms preprocess, 115.2ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)
Plant is in Center!

0: 384x640 1 person, 121.5ms
Speed: 3.0ms preprocess, 121.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Plant is in Center!

0: 384x640 1 person, 81.5ms
Speed: 2.2ms preprocess, 81.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Plant is in Center!

0: 384x640 1 person, 89.4ms
Speed: 2.0ms preprocess, 89.4ms inference, 2.0ms postprocess per image at shape (1, 3

In [ ]:
# For rpi4
import cv2
import cvzone
import math
from picamera.array import PiRGBArray
from picamera import PiCamera
from ultralytics import YOLO

# Initialize PiCamera
camera = PiCamera()
camera.resolution = (1280, 720)  # Set the resolution according to your preference
rawCapture = PiRGBArray(camera, size=(1280, 720))

# Initialize YOLO model
model = YOLO('yolov8n.pt')

# Define class names
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat", "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat", "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat", "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed", "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"]

alert_flag = False

# Main loop
for frame in camera.capture_continuous(rawCapture, format="bgr", use_video_port=True):
    # Grab the raw NumPy array representing the image
    img = frame.array
    
    # Object detection
    results = model(img, stream=True)
    for r in results:
        boxes = r.boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            w, h = x2 - x1, y2 - y1
            cvzone.cornerRect(img, (x1, y1, w, h))

            conf = math.ceil((box.conf[0] * 100)) / 100

            cls = box.cls[0]
            name = classNames[int(cls)]

            cvzone.putTextRect(img, f'{name} 'f'{conf}', (max(0, x1), max(35, y1)), scale=0.5)
            
            # Calculate the center region based on frame size
            frame_height, frame_width, _ = img.shape
            center_x_min = int(frame_width * 0.4)
            center_x_max = int(frame_width * 0.6)
            center_y_min = int(frame_height * 0.4)
            center_y_max = int(frame_height * 0.6)

            # Check if the object is near the dynamically calculated center
            if center_x_min < (x1 + x2) / 2 < center_x_max and center_y_min < (y1 + y2) / 2 < center_y_max:
                alert_flag = True
            else:
                alert_flag = False

    # Show alert if object in center
    if alert_flag:
        cv2.putText(img, "Object in Center!", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        print("Object in Center!")

    # Show the frame
    cv2.imshow("Image", img)

    # Exit if the user presses 'q'
    key = cv2.waitKey(1)
    if key == ord('q'):
        break

    # Clear the stream in preparation for the next frame
    rawCapture.truncate(0)

# Release the camera and close all OpenCV windows
camera.close()
cv2.destroyAllWindows()
